In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#Library Imports 
import pandas as pd
import tensorflow as tf
import pickle
import numpy as np
import random
import matplotlib.pyplot as plt
import keras
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score, precision_score, recall_score, f1_score
import scikitplot
from keras.models import Model, Sequential
from keras.layers import *
from keras.optimizers import Adam, SGD
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,EarlyStopping

print(keras.__version__)

Using TensorFlow backend.


2.3.1


In [ ]:
train_df = pd.read_csv('/kaggle/input/til2020/TIL_NLP_train_dataset.csv')
predict_df = pd.read_csv('/kaggle/input/til2020/TIL_NLP_test_dataset.csv')

In [ ]:
# Checking for null
print("There are ", train_df.isnull().sum().sum(), "null values in the dataset")

In [ ]:
labels = ['outwear','top','trousers','women dresses','women skirts']

for label in labels:
    print('Count:' + str(train_df[label].value_counts()[1]))

In [ ]:
reps = [5 if val==1 else 0 for val in train_df['women dresses']]
dresses_df = train_df.loc[np.repeat(train_df.index.values, reps)]
train_df = pd.concat([train_df,dresses_df.sample(n=2000)])

reps = [5 if val==1 else 0 for val in train_df['women skirts']]
skirts_df = train_df.loc[np.repeat(train_df.index.values, reps)]
train_df = pd.concat([train_df,skirts_df.sample(n=2000)])

In [ ]:
labels = ['outwear','top','trousers','women dresses','women skirts']

for label in labels:
    print('Count:' + str(train_df[label].value_counts()[1]))

In [ ]:
#Split Data
X_train, X_test, y_train, y_test = train_test_split(train_df.word_representation.values,
                                                    train_df[['outwear','top','trousers','women dresses','women skirts']].values,
                                                    test_size=0.2, 
                                                    stratify=train_df[['outwear','top','trousers','women dresses','women skirts']])

In [ ]:
#Instantiate tokenizer
tokenizer = text.Tokenizer(num_words=None)
max_len = 30
tokenizer.fit_on_texts(list(xtrain) + list(xval))

#Change train tests to sequences
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

#Change train sequences to pad
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

word_index = tokenizer.word_index

X_predict = list(predict_df["word_representation"])
X_predict = tokenizer.texts_to_sequences(X_predict)
X_predict = sequence.pad_sequences(X_predict, maxlen=max_len)

In [ ]:
import pickle

with open("/kaggle/input/til2020/word_embeddings.pkl", "rb" ) as p:
    word_embeddings = pickle.load(p)
    
embedding_matrix = np.zeros((len(word_index) + 1, 100)) # As each array for each word has 100 values
for word, i in word_index.items():         #presumably words we actually use in our dataset
    embedding_vector = word_embeddings.get(word) #get vectors for words we are using from Glove dataset
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
token_inputs = Input(shape=(maxlen,))

embedding_layer = Embedding(len(tokenizer.word_index) + 1, 
                            100, 
                            weights=[embedding_matrix], 
                            trainable=True)(token_inputs)

#Add Spatial Dropout 1D Layer
x = SpatialDropout1D(0.3)(embedding_layer)

#Original
# x = LSTM(64)(x)
# x = Dropout(0.7)(x)
# x = Dense(32, activation='relu')(x)        #Best so far: LSTM 64, Dropout 0.5, Dense 64

#Bidirectional LSTM
# x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
# x = GlobalMaxPool1D()(x)
# x = Dense(50, activation="relu")(x)
# x = Dropout(0.1)(x)

#Bidirectional LSTM with two dense layers
# x = Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3))(x)
# x = Dense(1024, activation='relu')(x)
# x = Dropout(0.8)(x)
# x = Dense(1024, activation='relu')(x)
# x = Dropout(0.8)(x)


#Additional LSTM + Dropout
# x = LSTM(128, return_sequences=True)(x)
# x = Dropout(0.5)(x)
# x = LSTM(64)(x)
# x = Dropout(0.5)(x)
# x = Dense(32, activation='relu')(x)

#GRU
# x = GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True)(x)
# x = GRU(300, dropout=0.3, recurrent_dropout=0.3)(x)
# x = Dense(1024, activation='relu')(x)
# x = Dropout(0.8)(x)
# x = Dense(1024, activation='relu')(x)
# x = Dropout(0.8)(x)

outputs = Dense(5, activation='sigmoid')(x)
model = Model(inputs=token_inputs, outputs=outputs)

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy', 'binary_crossentropy'])
#model.load_weights('/content/drive/My Drive/Colab Notebooks/TIL2020 Actual/jackpot_checkpoint')

model.summary()

In [ ]:
# Visualise model architecture
plot_model(model, to_file='model_plot4a.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Train model for N epochs. Vary as you desire
history = model.fit(X_train, 
                    y_train, 
                    batch_size=64, 
                    epochs=100, 
                    verbose=1, 
                    validation_data=(X_test, y_test),
                    callbacks = [
                        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr = 0.00001),
                        EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto'),
                        ModelCheckpoint(filepath='model-simple.h5',verbose = 1, save_best_only=True, mode = 'auto')
                   ])

#rename the filepath accordingly.

In [ ]:
# Print accuracy 

score = model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
probas = model.predict(X_test)

pred_indices = np.argmax(probas, axis=1)
classes = np.array(range(1, 6))
preds = classes[pred_indices]
print('Log loss: {}'.format(log_loss(classes[np.argmax(y_test, axis=1)], probas)))
print('Accuracy: {}'.format(accuracy_score(classes[np.argmax(y_test, axis=1)], preds)))
scikitplot.metrics.plot_confusion_matrix(classes[np.argmax(y_test, axis=1)], preds)

In [ ]:
%matplotlib inline
plt.style.use("ggplot")

# list all data in history
print(history.history.keys())

# summarize history for AUC
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
import xgboost as xgb
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import GridSearchCV

clf = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=4))

# You may need to use MultiLabelBinarizer to encode your variables from arrays [[x, y, z]] to a multilabel 
# format before training.
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(y_train)

clf.fit(X_train, y_train)

predictions = clf.predict_proba(X_test)

print('Log loss: {}'.format(log_loss(y_test, predictions)))

In [ ]:
predictions = model.predict(X_predict)
prediction_df = pd.DataFrame(predictions)
prediction_df.columns = ["outwear", "top", "trousers", "women dresses", "women skirts"]
prediction_df.index.name = 'id'
prediction_df = prediction_df.round().astype(int)
prediction_df.to_csv('predictions.csv')
prediction_df.head(10)

In [ ]:
# Save the weights
#model.save_weights('/content/drive/My Drive/Colab Notebooks/TIL2020 Actual/jackpot_checkpoint')

# Restore the weights
# model.load_weights('./checkpoints/my_checkpoint')

In [ ]:
kaggle competitions submit -c til2020 -f submission.csv -m "Message"